# Import

In [ ]:
# IMPORT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# balance par mois sur les credits en statut
cust = pd.read_csv('input/archive/olist_customers_dataset.csv',
                   delimiter=',', error_bad_lines=False, low_memory=False)
geo = pd.read_csv('input/archive/olist_geolocation_dataset.csv',
                  delimiter=',',
                  error_bad_lines=False,
                  low_memory=False)
order_item = pd.read_csv('input/archive/olist_order_items_dataset.csv',
                         delimiter=',',
                         error_bad_lines=False,
                         low_memory=False)
order_payment = pd.read_csv('input/archive/olist_order_payments_dataset.csv',
                            delimiter=',',
                            error_bad_lines=False,
                            low_memory=False)
order_review = pd.read_csv('input/archive/olist_order_reviews_dataset.csv',
                           delimiter=',',
                           error_bad_lines=False,
                           low_memory=False)
order = pd.read_csv('input/archive/olist_orders_dataset.csv',
                    delimiter=',',
                    error_bad_lines=False,
                    low_memory=False)
product = pd.read_csv('input/archive/olist_products_dataset.csv',
                      delimiter=',',
                      error_bad_lines=False,
                      low_memory=False)
seller = pd.read_csv('input/archive/olist_sellers_dataset.csv',
                     delimiter=',',
                     error_bad_lines=False,
                     low_memory=False)
product_cat = pd.read_csv('input/archive/' +
                          'product_category_name_translation.csv',
                          delimiter=',',
                          error_bad_lines=False,
                          low_memory=False)

In [ ]:
print('cust lignes/colonnes', cust.shape)
print('geo lignes/colonnes', geo.shape)
print('order_item lignes/colonnes', order_item.shape)
print('order_payment lignes/colonnes', order_payment.shape)
print('order_review lignes/colonnes', order_review.shape)
print('order lignes/colonnes', order.shape)
print('product lignes/colonnes', product.shape)
print('seller lignes/colonnes', seller.shape)
print('product_cat lignes/colonnes', product_cat.shape)

display(order)
display(order_item)
display(order_payment)
display(order_review)
display(product)
display(product_cat)
display(seller)
display(geo)
display(cust)

# Analyse produits commandés
* jointure product/product cat / order_item / seller

In [ ]:
# jointure produit et cat pour avoir la cat anglaise plus facile pour nous
import datetime as dt
product2 = pd.merge(product, product_cat, how='outer',
                    on='product_category_name')
# display(product2)

# champs inutiles
lis2dr = ['product_category_name',
          'product_name_lenght', 'product_description_lenght']
product2 = product2.drop(columns=lis2dr)


# fusion seller/order_item/product
product3 = order_item.set_index('seller_id').join(
    seller.set_index('seller_id'))
product3 = product3.reset_index().set_index(
    'product_id').join(product2.set_index('product_id'))

pd.set_option('max_columns', None)
# display(product3)
# display(product3.loc[product3.order_id=='e8fa22c3673b1dd17ea315021b1f0f61',:])
pd.set_option('max_columns', 10)
# order_item_id represente le numero de l'article dans la commande
# ex: 1er article 2eme article...

vc1 = pd.DataFrame(product3.value_counts('product_category_name_english'))
vc1.columns = ['val']
vc1['pct'] = vc1['val']/vc1['val'].sum()
pd.set_option('max_rows', None)
display(vc1)
pd.set_option('max_rows', 10)

vc2 = pd.DataFrame(product3.value_counts('seller_state'))
vc2.columns = ['val']
vc2['pct'] = vc2['val']/vc2['val'].sum()
# display(vc2)

vc3 = pd.DataFrame(product3.value_counts('seller_city'))
vc3.columns = ['val']
vc3['pct'] = vc3['val']/vc3['val'].sum()
# display(vc3)

plt.rcParams["figure.figsize"] = [16, 6]
plt.rcParams["figure.autolayout"] = True
f, axes = plt.subplots(1, 2)

axes[0].pie(vc2.loc[(vc2.pct >= 0.01)]['pct'],
            labels=vc2.loc[(vc2.pct >= 0.01)]['pct'].index,
            normalize=False,
            autopct='%1.1f%%',
            shadow=True,
            startangle=90)  # explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title('commandes: distribution des etats des vendeurs')

axes[1].pie(vc3.loc[(vc3.pct >= 0.01)]['pct'],
            labels=vc3.loc[(vc3.pct >= 0.01)]['pct'].index,
            normalize=False, autopct='%1.1f%%',
            shadow=True, startangle=90)  # explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title('commandes: distribution des villes des vendeurs')
plt.show()


#################################################
# calcul des montant par type de prduit
vc4 = product3.groupby(
    by="product_category_name_english").agg({'price': ['sum']})
vc4.columns = vc4.columns.droplevel(0)
vc4.columns = ['ca']
vc4 = vc4.sort_values('ca', ascending=False)
vc4['pct'] = vc4.ca/vc4.ca.sum()

# display(vc4)
f, axes = plt.subplots(1, 2)

axes[0].pie(vc1.loc[(vc1.pct >= 0.01)]['pct'],
            labels=vc1.loc[(vc1.pct >= 0.01)]['pct'].index,
            normalize=False,
            autopct='%1.1f%%',
            shadow=True,
            startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title('commandes: distribution des catgeories de produits')

axes[1].pie(vc4.loc[(vc4.pct >= 0.01)]['pct'],
            labels=vc4.loc[(vc4.pct >= 0.01)]['pct'].index,
            normalize=False,
            autopct='%1.1f%%',
            shadow=True,
            startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title('ca: distribution des catgeories de produits')

#################################################
f, axes = plt.subplots(1, 1)
ax = sns.boxplot(x='product_category_name_english', y='price',
                 data=product3, showmeans=True, showfliers=False)
ax.title.set_text("prix moyen par categorie")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

plt.show

#################################################
vc4['ca_rank'] = vc4.pct.rank(method='max', ascending=False)
# display(vc4)

product4 = product3.reset_index()
product4 = product4.set_index('product_category_name_english').join(vc4)
# display(product4.loc[product4.pct>=0.01,:].sort_values('ca_rank',ascending=True).reset_index())

f, axes = plt.subplots(1, 1)
ax = sns.boxplot(x='product_category_name_english', y='price',
                 data=product4.loc[product4.pct >= 0.01, :].sort_values(
                     'ca_rank', ascending=True).reset_index(),
                 showmeans=True,
                 showfliers=False)
ax.title.set_text(
    "prix moyen par categorie (celles qui font plus de 1% du CA total)")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show

f, axes = plt.subplots(1, 1)
ax = sns.boxplot(x='product_category_name_english',
                 y='pct',
                 data=product4.loc[product4.pct >= 0.01, :].sort_values(
                     'ca_rank', ascending=True).reset_index(),
                 showmeans=True,
                 showfliers=False)
ax.title.set_text(
    "CA moyen par categorie (celles qui font plus de 1% du CA total)")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show

#################################################
# "datetime"
# attention les plot ont du mal avec les types timedelta
# -> les changer en float ou avec total_seconds
product4.shipping_limit_date = pd.to_datetime(
    product4.shipping_limit_date, errors='coerce')
product4['init_date'] = dt.datetime(2016, 1, 1, 0, 0, 0)

product4['difdate'] = product4.shipping_limit_date-product4['init_date']
# operator // pour division entiere
product4['difdate'] = (product4['difdate'].dt.total_seconds()/3600/24)//7


product4['year'] = product4.shipping_limit_date.dt.isocalendar().year
product4['mois'] = product4.shipping_limit_date.dt.month
product4['semaine'] = product4.shipping_limit_date.dt.isocalendar().week
product4 = product4.loc[(product4.year < 2020), :]
# display(product4.sort_values('shipping_limit_date'))

histprod4 = pd.DataFrame(product4.value_counts(
    'difdate')).sort_values('difdate')
histprod4.columns = ['nbc']

ax0 = plt.subplot(111)
sns.barplot(x=histprod4.index, y='nbc', data=histprod4, color='blue', ax=ax0)
ax0.set_title(
    "nb de commandes par semaine" +
    " sur l'échantillon : semaine 1 commence le 01/01/2016")
plt.xticks(np.arange(0, 105, 5).tolist())
plt.show()
#######################################################
histprod4_ca = product4.groupby(by="difdate").agg({'price': ['sum']})
histprod4_ca.columns = histprod4_ca.columns.droplevel(0)
# display(histprod4_ca)

ax0 = plt.subplot(111)
sns.barplot(x=histprod4_ca.index, y='sum',
            data=histprod4_ca, color='blue', ax=ax0)
ax0.set_title(
    "CA par semaine sur l'échantillon : semaine 1 commence le 01/01/2016")
plt.xticks(np.arange(0, 105, 5).tolist())
plt.show()

In [ ]:
product5 = product4.reset_index()
pd.set_option('max_columns', None)
display(product5)
pd.set_option('max_columns', 10)

# analyse order_payment

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
display(order_payment.payment_type.value_counts(normalize=True))
display(order_payment.payment_installments.value_counts().head(10))
order_payment.loc[order_payment.payment_sequential > 1, :]
order_payment.loc[order_payment.order_id ==
                  'fedcd9f7ccdc8cba3a18defedd1a5547', :].sort_values(
    'payment_sequential')
display(order_payment.loc[order_payment.payment_installments == 3, :])
order_payment.loc[order_payment.order_id ==
                  '1dcf0c8cd36ffaf57784fbdc90079310', :].sort_values(
    'payment_sequential')


plt.figure(figsize=(4, 4))
ax = order_payment.payment_type.value_counts(normalize=True).plot.barh(
    color="#8d19a9",
    title='Pourcentage de type de paiement',
    xlabel='pourcentage de type de paiement')
ax.grid(zorder=0)
plt.show()

## aggregation des order_payments par order_id
* payment_sequential count/max
* payment_installments min/mean/max
* payment_value sum/min/mean/max
* 'payment_type' nb et pct du type de paiement (cash/credit card/debt card/ not defined / voucher
* payment value par type de paiement

* si cash: commande payée entierement en cash
* si debit commande payée entierement en debit
* si credit 3% des commandes payées en plus avec des voucher
* seuls les credits et mix credit voucher peuvent avoir plusieurs paiements

In [ ]:
# payment_sequential: count et max doivent donner la meme chose
pay1 = order_payment.groupby(by="order_id").agg({
    'payment_sequential': ['count', 'max'],
    'payment_installments': ['min', 'mean', 'max'],
    'payment_value': ['sum', 'min', 'mean', 'max']})
pay1.columns = pay1.columns.droplevel(0)
pay1.columns = ['pay_seq_count', 'pay_seq_max',
                'pay_ins_min', 'pay_ins_mean',
                'pay_ins_max', 'pay_val_sum',
                'pay_val_min', 'pay_val_mean',
                'pay_val_max']
# seulement 80 commandes avec sequence count different de sequence max
pay1.loc[pay1.pay_seq_count != pay1.pay_seq_max, :]

pay2 = order_payment.groupby(
    'order_id')['payment_type'].value_counts().unstack()
pay2.columns = ['cash', 'creditc', 'debitc', 'nd', 'voucher']
pay3 = order_payment.groupby(
    'order_id')['payment_type'].value_counts(normalize=True).unstack()
pay3.columns = ['cash_pct', 'creditc_pct',
                'debitc_pct', 'nd_pct', 'voucher_pct']
pay2 = pay2.join(pay3)
pay2 = pay2.fillna(0)
pay1 = pay1.join(pay2)


# calcul des montant par type de paiement
pay4 = order_payment.loc[
    order_payment.payment_type == 'credit_card', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay4.columns = pay4.columns.droplevel(0)
pay4.columns = ['pay_val_creditc']
pay5 = order_payment.loc[
    order_payment.payment_type == 'debit_card', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay5.columns = pay5.columns.droplevel(0)
pay5.columns = ['pay_val_debitc']
pay6 = order_payment.loc[
    order_payment.payment_type == 'boleto', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay6.columns = pay6.columns.droplevel(0)
pay6.columns = ['pay_val_cash']
pay7 = order_payment.loc[
    order_payment.payment_type == 'voucher', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay7.columns = pay7.columns.droplevel(0)
pay7.columns = ['pay_val_voucher']
pay8 = order_payment.loc[
    order_payment.payment_type == 'not_defined', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay8.columns = pay8.columns.droplevel(0)
pay8.columns = ['pay_val_nd']

print('pourcentage des paiements faits en credit card:',
      pay1.creditc_pct.mean())
print('pourcentage des paiements faits en debit card:',
      pay1.debitc_pct.mean())
print('pourcentage des paiements faits en cash:',
      pay1.cash_pct.mean())
print('pourcentage des paiements faits en voucher:',
      pay1.voucher_pct.mean())

pay1 = pay1.join(pay4)
pay1 = pay1.join(pay5)
pay1 = pay1.join(pay6)
pay1 = pay1.join(pay7)
pay1 = pay1.join(pay8)

pay1[['pay_val_creditc', 'pay_val_debitc',
      'pay_val_cash', 'pay_val_voucher', 'pay_val_nd']] = pay1[[
          'pay_val_creditc', 'pay_val_debitc',
          'pay_val_cash', 'pay_val_voucher', 'pay_val_nd']].fillna(0)
pay1['pay_val_pct_creditc'] = pay1['pay_val_creditc']/pay1['pay_val_sum']
pay1['pay_val_pct_debitc'] = pay1['pay_val_debitc']/pay1['pay_val_sum']
pay1['pay_val_pct_cash'] = pay1['pay_val_cash']/pay1['pay_val_sum']
pay1['pay_val_pct_voucher'] = pay1['pay_val_voucher']/pay1['pay_val_sum']
pay1['pay_val_pct_nd'] = pay1['pay_val_nd']/pay1['pay_val_sum']

# petit nettoyage
pay1 = pay1.loc[pay1.pay_seq_count ==
                pay1.pay_seq_max, :]  # on retire 80 lignes
pay1 = pay1.loc[pay1.pay_val_pct_nd != 1, :]
pay1 = pay1.drop(columns=['pay_seq_max', 'pay_val_nd', 'pay_val_pct_nd'])

display(pay1.pay_val_pct_cash.value_counts())
display(pay1.pay_val_pct_voucher.value_counts())
display(pay1.pay_val_pct_debitc.value_counts())
display(pay1.pay_val_pct_creditc.value_counts())

pay1.loc[pay1.pay_val_pct_cash == 1, ['pay_type']] = 'cash'
pay1.loc[pay1.pay_val_pct_debitc == 1, ['pay_type']] = 'debit'
pay1.loc[((pay1.pay_val_pct_creditc == 1) & (
    pay1.pay_val_pct_voucher == 0)), ['pay_type']] = 'credit'
pay1.loc[((pay1.pay_val_pct_voucher == 1) & (
    pay1.pay_val_pct_creditc == 0)), ['pay_type']] = 'voucher'
pay1.loc[((pay1.pay_val_pct_voucher > 0) & (pay1.pay_val_pct_creditc > 0)), [
    'pay_type']] = 'mix credit voucher'

# autre nettoyage pour données inutiles
pay1 = pay1.drop(columns=['pay_val_pct_creditc',
                 'pay_val_pct_debitc', 'pay_val_pct_cash'])
pay1 = pay1.drop(columns=['pay_val_creditc', 'pay_val_debitc', 'pay_val_cash'])
pay1 = pay1.drop(columns=['cash_pct', 'creditc_pct', 'debitc_pct', 'nd_pct'])
pay1 = pay1.drop(columns=['cash', 'creditc', 'debitc', 'nd'])

pd.set_option('max_columns', None)
display(pay1.loc[pay1.pay_ins_min != pay1.pay_ins_max, :])
pd.set_option('max_columns', 10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
display(pay1.pay_type.value_counts(normalize=True))


plt.rcParams["figure.figsize"] = [16, 5]
plt.rcParams["figure.autolayout"] = True

f, axes = plt.subplots(1, 3)
sns.countplot(x='pay_type', data=pay1, ax=axes[0])
sns.boxplot(x='pay_type', y='pay_val_sum',
            data=pay1, showmeans=True, ax=axes[1])
sns.boxplot(x='pay_type', y='pay_val_sum', data=pay1,
            showfliers=False, showmeans=True, ax=axes[2])
axes[0].set_title('type de payment')
axes[1].set_title('somme par commande')
axes[2].set_title('zoom somme par commande')
plt.show()

f, axes = plt.subplots(1, 3)
sns.boxplot(x='pay_type', y='pay_val_pct_voucher',
            data=pay1.loc[pay1.pay_type == 'mix credit voucher', :],
            showmeans=True,
            ax=axes[0])
sns.boxplot(x='pay_type', y='pay_ins_max',
            data=pay1, showmeans=True, ax=axes[1])
sns.boxplot(x='pay_type', y='pay_ins_max', data=pay1,
            showfliers=False, showmeans=True, ax=axes[2])
axes[0].set_title('pct de voucher pour paiement mixte')
axes[1].set_title('nb pay par commande')
axes[2].set_title('zoom nb pay par commande')
plt.show()

# jointure order et order_payment et analyse d'order

In [ ]:
# on fusionne avec order
order1 = order.set_index('order_id')
order1 = order1.join(pay1)
# "datetime"
order1.order_purchase_timestamp = pd.to_datetime(
    order1.order_purchase_timestamp, errors='coerce')
order1.order_approved_at = pd.to_datetime(
    order1.order_approved_at, errors='coerce')
order1.order_delivered_carrier_date = pd.to_datetime(
    order1.order_delivered_carrier_date, errors='coerce')
order1.order_delivered_customer_date = pd.to_datetime(
    order1.order_delivered_customer_date, errors='coerce')
order1.order_estimated_delivery_date = pd.to_datetime(
    order1.order_estimated_delivery_date, errors='coerce')

# difference de temps (c est vraimeent histoire 2)
order1['tim2approv'] = order1['order_approved_at'] - \
    order1['order_purchase_timestamp']
order1['tim2carrier'] = order1['order_delivered_carrier_date'] - \
    order1['order_approved_at']
order1['tim2deliv'] = order1['order_delivered_customer_date'] - \
    order1['order_delivered_carrier_date']
order1['tim2estim'] = order1['order_estimated_delivery_date'] - \
    order1['order_delivered_customer_date']

display(order1.order_status.value_counts(normalize=True))

# test doublons sur clés


def doublonrem(datain, datanam, keynam):
    """enleve les doublons."""
    temp = datain.copy()
    temp2 = temp.drop_duplicates(subset=keynam, keep='last', inplace=False)
    # inplace -> renvoie une table filtrée
    # ou modifie directement la table de base sans rien renvoyer
    print('extraction de ', temp2.shape[0]-datain.shape[0],
          'doublons de la table ', datanam, ' basée sur la variable ', keynam)
    return temp2
# order2=doublonrem(order1.reset_index(),'order','order_id')


order1 = order1.sort_values('order_purchase_timestamp')
pd.set_option('max_columns', None)
display(order1)
pd.set_option('max_columns', 10)

In [ ]:
import datetime as dt
print(order1.order_purchase_timestamp.dtypes)
print(order1.tim2approv.dtypes)

# attention les plot ont du mal avec les types timedelta
# -> les changer en float ou avec total_seconds
order1.tim2approv = order1.tim2approv.dt.total_seconds()/3600/24
order1.tim2carrier = order1.tim2carrier.dt.total_seconds()/3600/24
order1.tim2deliv = order1.tim2deliv.dt.total_seconds()/3600/24
order1.tim2estim = order1.tim2estim.dt.total_seconds()/3600/24

print(order1.tim2approv.dtypes)

plt.rcParams["figure.figsize"] = [16, 4]
plt.rcParams["figure.autolayout"] = True

f, axes = plt.subplots(1, 2)
sns.scatterplot(data=order1, x="order_purchase_timestamp",
                y="tim2approv", ax=axes[0])
sns.scatterplot(data=order1, x="order_purchase_timestamp",
                y="tim2carrier", ax=axes[1])
axes[0].set_title('nombre de jours avant validation de la commande')
axes[1].set_title('nombre de jours entre validation et transporteur')
plt.show()
f, axes = plt.subplots(1, 2)
sns.scatterplot(data=order1, x="order_purchase_timestamp",
                y="tim2deliv", ax=axes[0])
sns.scatterplot(data=order1, x="order_purchase_timestamp",
                y="tim2estim", ax=axes[1])
axes[0].set_title('nombre de jours entre transporteur et livraison')
axes[1].set_title("nombre de jours d'avance sur la date de livraison estimée")
plt.show()
f, axes = plt.subplots(1, 2)
sns.boxplot(x='order_status', y='tim2approv',
            data=order1, showmeans=True, ax=axes[0])
sns.boxplot(x='order_status', y='tim2approv', data=order1,
            showfliers=False, showmeans=True, ax=axes[1])
axes[0].set_title('nombre de jours avant validation de la commande par statut')
axes[1].set_title(
    "nombre de jours avant validation de la commande par statut" +
    " - zoom sans outliers")
plt.show()
f, axes = plt.subplots(1, 2)
sns.boxplot(x='order_status', y='tim2carrier',
            data=order1, showmeans=True, ax=axes[0])
sns.boxplot(x='order_status', y='tim2carrier', data=order1,
            showfliers=False, showmeans=True, ax=axes[1])
axes[0].set_title(
    'nombre de jours entre validation et transporteur par statut')
axes[1].set_title(
    "nombre de jours entre validation et transporteur par statut" +
    "- zoom sans outliers")
plt.show()
f, axes = plt.subplots(1, 2)
sns.boxplot(x='order_status', y='tim2deliv',
            data=order1, showmeans=True, ax=axes[0])
sns.boxplot(x='order_status', y='tim2deliv', data=order1,
            showfliers=False, showmeans=True, ax=axes[1])
axes[0].set_title("nombre de jours entre transporteur et livraison" +
                  " par statut")
axes[1].set_title(
    "nombre de jours entre transporteur et livraison par statut" +
    " - zoom sans outliers")
plt.show()
f, axes = plt.subplots(1, 2)
sns.boxplot(x='order_status', y='tim2estim',
            data=order1, showmeans=True, ax=axes[0])
sns.boxplot(x='order_status', y='tim2estim', data=order1,
            showfliers=False, showmeans=True, ax=axes[1])
axes[0].set_title(
    "nombre de jours d'avance sur la date de livraison estimée par statut")
axes[1].set_title(
    "nombre de jours d'avance sur la date de livraison estimée " +
    "par statut - zoom sans outliers")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
orderhist = order1[['order_purchase_timestamp', 'pay_val_sum']].copy()
orderhist['pay_val_cumsum'] = orderhist.pay_val_sum.cumsum()
orderhist['mois'] = orderhist.order_purchase_timestamp.dt.month
orderhist['semaine'] = orderhist.order_purchase_timestamp.dt.isocalendar().week
# display(orderhist)
# sns.relplot(data=orderhist, x="order_purchase_timestamp",
# y="pay_val_cumsum", kind="line")

orderseason1 = orderhist.groupby(by="mois").agg(
    {'pay_val_sum': ['count', 'mean', 'median']})
orderseason1.columns = orderseason1.columns.droplevel(0)
orderseason2 = orderhist.groupby(by="semaine").agg(
    {'pay_val_sum': ['count', 'mean', 'median']})
orderseason2.columns = orderseason2.columns.droplevel(0)
# display(orderseason1)


# chose tres importante
# pour aligner les x avec un double y utiliser la fonction
# ax.get_xticks() pour le x du second axe
#

ax0 = plt.subplot(121)
ax1 = ax0.twinx()  # permet le double axe
ax2 = plt.subplot(122)
ax3 = ax2.twinx()

sns.barplot(x=orderseason1.index, y='count',
            data=orderseason1, color='blue', ax=ax0)
sns.lineplot(data=orderseason1, x=ax0.get_xticks(),
             y=orderseason1['mean'], marker='s', legend="auto", ax=ax1)
sns.lineplot(data=orderseason1, x=ax0.get_xticks(),
             y=orderseason1['median'], marker='s', legend="auto", ax=ax1)

sns.barplot(x=orderseason2.index, y='count',
            data=orderseason2, color='blue', ax=ax2)
sns.lineplot(data=orderseason2, x=ax2.get_xticks(),
             y=orderseason2['mean'], marker='s', legend='brief', ax=ax3)
sns.lineplot(data=orderseason2, x=ax2.get_xticks(),
             y=orderseason2['median'], marker='s', legend='brief', ax=ax3)

ax1.set(ylabel='median(o)/mean(b)')
ax3.set(ylabel='median(o)/mean(b)')
ax0.set_title('panier moyen / median vs nb de commandes par mois')
ax2.set_title('panier moyen / median vs nb de commandes par semaine')
plt.xticks(np.arange(0, 52, step=2))
plt.show()

In [ ]:
import matplotlib.image as img
ordercust = order1.copy()
ordercust = ordercust.reset_index()
# fusion order et customer
ordercust1 = pd.merge(ordercust, cust, how='outer', on='customer_id')
# passage en dummies pour le type de paiement
ordercust2 = pd.get_dummies(data=ordercust1, columns=['pay_type'])
pd.set_option('max_columns', None)
# display(ordercust)#99441 commandes/lignes
# display(cust) #99441 customer id/ lignes (1 par commande en fait)
# -> c 'est le customer unique id qui compte'
# display(ordercust2)
pd.set_option('max_columns', 10)

# aggregation par customer_unique _id
ordercust3 = ordercust2.groupby(by="customer_unique_id").agg({
    'pay_val_sum': ['count', 'sum', 'mean', 'median'],
    'pay_type_cash': ['mean'],
    'pay_type_debit': ['mean'],
    'pay_type_credit': ['mean'],
    'pay_type_mix credit voucher': ['mean'],
    'pay_type_voucher': ['mean']})

ordercust3.columns = ordercust3.columns.droplevel(0)
ordercust3.columns = ['pay_count', 'pay_sum', 'pay_mean',
                      'pay_median', 'cash', 'debit', 'credit', 'mix', 'voucher']
# display(ordercust3.sort_values('pay_count'))

######################################################
ordercust4 = ordercust3.value_counts('pay_count', normalize=True)
display(ordercust4)
print("97% des clients n'ont passé qu'une seule commande")

f, axes = plt.subplots(1, 2)

axes[0].pie(ordercust3.value_counts('pay_count',
                                    normalize=True).head(2),
            labels=ordercust3.value_counts('pay_count').head(2).index,
            normalize=False, autopct='%1.1f%%',
            shadow=True, startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title('nombre de commandes par client')

ordercust3['pay_type'] = 'mix'
ordercust3.loc[(ordercust3.cash+ordercust3.debit == 1),
               ['pay_type']] = 'cash_debit'
ordercust3.loc[(ordercust3.credit+ordercust3.voucher +
                ordercust3.mix == 1), ['pay_type']] = 'credit_voucher'

ordercust5 = ordercust3.value_counts('pay_type', normalize=True)
display(ordercust5)
print('tres peu de mix credit/cash')

f, axes = plt.subplots(1, 2)
sns.scatterplot(data=ordercust3, x='pay_count', y='pay_sum',
                hue='pay_type', alpha=0.5, ax=axes[0])
sns.scatterplot(data=ordercust3, x='pay_count', y='pay_mean',
                hue='pay_type', alpha=0.5, ax=axes[1])
axes[0].set_title(
    "CA par nombre d'achat par client et par type de paiement du client")
axes[1].set_title(
    "CA moyen par nombre d'achat par client et par type de paiement du client")
plt.show

f, axes = plt.subplots(1, 2)
sns.boxplot(x='pay_type', y='pay_mean',
            data=ordercust3.loc[ordercust3.pay_type != 'mix', :],
            showmeans=True, showfliers=False, ax=axes[0])
sns.boxplot(x='pay_type', y='pay_sum',
            data=ordercust3.loc[ordercust3.pay_type != 'mix', :],
            showmeans=True, showfliers=False, ax=axes[1])
axes[0].set_title("CA par type de paiement du client")
axes[1].set_title("CA moyen par type de paiement du client")
plt.show

######################################################
idcp = ordercust2[['customer_unique_id', 'customer_zip_code_prefix',
                   'customer_city', 'customer_state']].copy()
idcp2 = doublonrem(idcp, 'unique id', 'customer_unique_id')

idcp2['customer_city'] = idcp2['customer_city'].str.replace('ã', 'a')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('í', 'i')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('é', 'e')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('ê', 'e')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('ô', 'o')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('á', 'a')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('ú', 'u')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('â', 'a')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('ó', 'o')
idcp2['customer_city'] = idcp2['customer_city'].str.replace("t'", 't')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('õ', 'o')
idcp2['customer_city'] = idcp2['customer_city'].str.replace('ç', 'c')
idcp2['customer_city'] = idcp2['customer_city'].str.replace("d'", "d ")
idcp2['customer_city'] = idcp2['customer_city'].str.replace(
    "embu", 'embu das artes')
idcp2['customer_city'] = idcp2['customer_city'].str.replace(
    "das artes das artes", "das artes")
idcp2['customer_city'] = idcp2['customer_city'].str.replace("-", ' ')

# fusion avec la base geo
geo1 = geo.copy()

# nettoyage
# on enleve les positions hors du bresil aberrantes et on fixe le sproblemes d'orthographe
geo1.loc[((geo1.geolocation_lng > -28) | (geo1.geolocation_lng < -75) |
          (geo1.geolocation_lat > 5) | (geo1.geolocation_lat < -34)),
         ['geolocation_lat', 'geolocation_lng']] = np.nan
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ã', 'a')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('í', 'i')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('é', 'e')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ê', 'e')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ô', 'o')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('á', 'a')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ú', 'u')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('â', 'a')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ó', 'o')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace("t'", 't')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('õ', 'o')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ç', 'c')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace("d'", "d ")
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "embu", 'embu das artes')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "das artes das artes", "das artes")
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace("-", ' ')

geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "piumhi", 'piumhii')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "santa terezinha", 'santa teresinha')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "belem de sao francisco", 'belem do sao francisco')


geo1['geolocation_city'] = geo1['geolocation_city'].str.lower()

# aggregation des données geo par zip
geo1_a = geo1.groupby(by="geolocation_zip_code_prefix").agg(
    {'geolocation_lat': ['mean'], 'geolocation_lng': ['mean']})
geo1_a.columns = geo1_a.columns.droplevel(0)
geo1_a.columns = ['lat_zip', 'lng_zip']

# aggregation des données geo par city
geo1_b = geo1.groupby(by="geolocation_city").agg(
    {'geolocation_lat': ['mean'], 'geolocation_lng': ['mean']})
geo1_b.columns = geo1_b.columns.droplevel(0)
geo1_b.columns = ['lat_city', 'lng_city']

# aggregation des données geo par state
geo1_c = geo1.groupby(by="geolocation_state").agg(
    {'geolocation_lat': ['mean'], 'geolocation_lng': ['mean']})
geo1_c.columns = geo1_c.columns.droplevel(0)
geo1_c.columns = ['lat_state', 'lng_state']


# fusion city state et donnes geo moyennees
# on vire les doublons
geo2 = doublonrem(geo1, 'geo', 'geolocation_zip_code_prefix')
geo2 = geo2.loc[:, ['geolocation_zip_code_prefix', 'geolocation_city',
                    'geolocation_state']].set_index('geolocation_zip_code_prefix')
geo2 = geo2.join(geo1_a)

fus1 = idcp2.set_index('customer_zip_code_prefix').join(geo2)
fus1 = fus1.reset_index()
fus1.rename(columns={'index': 'customer_zip_code_prefix'}, inplace=True)

fus2 = fus1.set_index('customer_city').join(geo1_b)
fus2 = fus2.reset_index()
fus2.rename(columns={'index': 'customer_city'}, inplace=True)

fus3 = fus2.set_index('customer_state').join(geo1_c)
fus3 = fus3.reset_index()
fus3.rename(columns={'index': 'customer_state'}, inplace=True)

fus3[['lat_city_fill', 'lng_city_fill']] = fus3[['lat_city', 'lng_city']]
fus3.loc[(fus3.lat_city.isna()), ['lat_city_fill']] = fus3['lat_state']
fus3.loc[(fus3.lng_city.isna()), ['lng_city_fill']] = fus3['lng_state']

fus3 = fus3.set_index('customer_unique_id')
ordercustgeo = ordercust3.join(fus3)

#################################################
#################################################
display(ordercustgeo.value_counts('geolocation_state', normalize=True))
display(ordercustgeo.value_counts('geolocation_city', normalize=True))

f, axes = plt.subplots(1, 2)

axes[0].pie(ordercustgeo.value_counts('geolocation_state',
                                      normalize=True).head(10),
            labels=ordercustgeo.value_counts(
                'geolocation_state').head(10).index,
            normalize=False, autopct='%1.1f%%',
            shadow=True, startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title(
    'distribution des clients par etat (les 10 plus représentés)')

axes[1].pie(ordercustgeo.value_counts('geolocation_city',
                                      normalize=True).head(10),
            labels=ordercustgeo.value_counts(
                'geolocation_city').head(10).index,
            normalize=False, autopct='%1.1f%%', shadow=True, startangle=90)
# explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title(
    'distribution des clients par ville (les 10 plus representées)')
plt.show()

#################################################
test = pd.DataFrame(ordercustgeo.value_counts(
    'customer_state', normalize=True))
test.columns = ['cust_pct']
test['cust_state_rank'] = test['cust_pct'].rank(method='max', ascending=False)

ordercustgeo2 = ordercustgeo.reset_index()
ordercustgeo2.rename(columns={'index': 'customer_unique_id'}, inplace=True)
ordercustgeo2 = ordercustgeo2.set_index('customer_state')
ordercustgeo2 = ordercustgeo2.join(test['cust_state_rank'])
ordercustgeo2 = ordercustgeo2.reset_index()
ordercustgeo2.rename(columns={'index': 'customer_state'}, inplace=True)
ordercustgeo2 = ordercustgeo2.set_index('customer_unique_id')

# display(test)
# display(ordercustgeo2.sort_values('cust_state_rank'))

f, axes = plt.subplots(1, 1)
ax = sns.boxplot(x='customer_state', y='pay_mean',
                 data=ordercustgeo2.loc[(ordercustgeo2.cust_state_rank <= 10) & (
                     ordercustgeo2.pay_type != 'mix')].sort_values([
                         'cust_state_rank', 'pay_type']),
                 hue='pay_type',
                 showmeans=True,
                 showfliers=False)
ax.title.set_text(
    "panier moyen par etat (les 10 plus representés) et par type de paiement")
plt.show

#################################################
test = pd.DataFrame(ordercustgeo.value_counts('customer_city', normalize=True))
test.columns = ['cust_pct']
test['cust_city_rank'] = test['cust_pct'].rank(method='max', ascending=False)

ordercustgeo3 = ordercustgeo.reset_index()
ordercustgeo3.rename(columns={'index': 'customer_unique_id'}, inplace=True)
ordercustgeo3 = ordercustgeo3.set_index('customer_city')
ordercustgeo3 = ordercustgeo3.join(test['cust_city_rank'])
ordercustgeo3 = ordercustgeo3.reset_index()
ordercustgeo3.rename(columns={'index': 'customer_city'}, inplace=True)
ordercustgeo3 = ordercustgeo3.set_index('customer_unique_id')

# display(test)
# display(ordercustgeo3.sort_values('cust_city_rank'))

f, axes = plt.subplots(1, 1)
ax = sns.boxplot(x='customer_city', y='pay_mean',
                 data=ordercustgeo3.loc[(ordercustgeo3.cust_city_rank <= 10) & (
                     ordercustgeo3.pay_type != 'mix')].sort_values([
                         'cust_city_rank', 'pay_type']),
                 hue='pay_type',
                 showmeans=True,
                 showfliers=False)
ax.title.set_text("panier moyen par ville (les 10 plus représentées)")
plt.show

#################################################

plt.rcParams["figure.figsize"] = [16, 6]
plt.rcParams["figure.autolayout"] = True

ordercustgeo_aggstate = ordercustgeo.groupby(
    by="customer_state").agg({'pay_sum': ['sum']})
ordercustgeo_aggstate.columns = ordercustgeo_aggstate.columns.droplevel(0)
ordercustgeo_aggstate.columns = ['ca']
ordercustgeo_aggstate['ca_pct'] = ordercustgeo_aggstate.ca / \
    ordercustgeo_aggstate.ca.sum()

ordercustgeo_aggcity = ordercustgeo.groupby(
    by="customer_city").agg({'pay_sum': ['sum']})
ordercustgeo_aggcity.columns = ordercustgeo_aggcity.columns.droplevel(0)
ordercustgeo_aggcity.columns = ['ca']
ordercustgeo_aggcity['ca_pct'] = ordercustgeo_aggcity.ca / \
    ordercustgeo_aggcity.ca.sum()


# display(ordercustgeo_aggstate.sort_values('ca',ascending=False))
# display(ordercustgeo.value_counts('geolocation_state',normalize=True))
# display(ordercustgeo.value_counts('geolocation_city',normalize=True))

f, axes = plt.subplots(1, 2)

axes[0].pie(ordercustgeo_aggstate.sort_values('ca',
                                              ascending=False).ca_pct.head(10),
            labels=ordercustgeo_aggstate.sort_values(
                'ca', ascending=False).head(10).index,
            normalize=False, autopct='%1.1f%%',
            shadow=True,
            startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title('distribution des CA par etat (les 10 plus représentés)')

axes[1].pie(ordercustgeo_aggcity.sort_values('ca',
                                             ascending=False).ca_pct.head(10),
            labels=ordercustgeo_aggcity.sort_values(
                'ca', ascending=False).head(10).index,
            normalize=False, autopct='%1.1f%%',
            shadow=True,
            startangle=90)  # explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title('distribution des CA par ville (les 10 plus representées)')
plt.show()
####################################################################
brazil_img = img.imread('input/brazil.png')

###################################################
wesh1 = ordercustgeo_aggstate.join(doublonrem(ordercustgeo[[
    'customer_state',
    'lat_state',
    'lng_state']].set_index('customer_state'), 'geo', 'lat_state'))
wesh2 = ordercustgeo_aggcity.join(doublonrem(ordercustgeo[[
                                  'customer_city', 'lat_city_fill',
                                  'lng_city_fill']].set_index('customer_city'),
    'geo', 'lat_city_fill'))


f, axes = plt.subplots(1, 2)
sns.scatterplot(data=wesh1, x='lng_state', y='lat_state',
                size="ca_pct", hue="ca_pct", sizes=(40, 400),
                alpha=1, ax=axes[0])
sns.scatterplot(data=wesh2, x="lng_city_fill", y="lat_city_fill",
                size="ca_pct", hue="ca_pct", sizes=(40, 400),
                alpha=1, ax=axes[1])

axes[0].set_title('CA par etat')
axes[1].set_title('CA par villes')

im0 = axes[0].imshow(brazil_img, extent=[-75, -28, -34, 5],
                     alpha=0.8, aspect='auto')
im1 = axes[1].imshow(brazil_img, extent=[-75, -28, -34, 5],
                     alpha=0.8, aspect='auto')

plt.show()


####################################################################
ordercustgeo_aggstate = ordercustgeo.loc[ordercustgeo.pay_type == 'cash_debit',
                                         :].groupby(
    by="customer_state").agg({'pay_sum': ['sum']})
ordercustgeo_aggstate.columns = ordercustgeo_aggstate.columns.droplevel(0)
ordercustgeo_aggstate.columns = ['ca']
ordercustgeo_aggstate['ca_pct'] = ordercustgeo_aggstate.ca / \
    ordercustgeo_aggstate.ca.sum()

ordercustgeo_aggcity = ordercustgeo.loc[ordercustgeo.pay_type == 'cash_debit',
                                        :].groupby(
    by="customer_city").agg({'pay_sum': ['sum']})
ordercustgeo_aggcity.columns = ordercustgeo_aggcity.columns.droplevel(0)
ordercustgeo_aggcity.columns = ['ca']
ordercustgeo_aggcity['ca_pct'] = ordercustgeo_aggcity.ca / \
    ordercustgeo_aggcity.ca.sum()


# display(ordercustgeo_aggstate.sort_values('ca',ascending=False))
# display(ordercustgeo.value_counts('geolocation_state',normalize=True))
# display(ordercustgeo.value_counts('geolocation_city',normalize=True))

f, axes = plt.subplots(1, 2)

axes[0].pie(ordercustgeo_aggstate.sort_values('ca', ascending=False).ca_pct.head(10),
            labels=ordercustgeo_aggstate.sort_values(
    'ca', ascending=False).head(10).index, normalize=False, autopct='%1.1f%%',
    shadow=True, startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title(
    'distribution des CA (cash/debit) par etat (les 10 plus représentés)')

axes[1].pie(ordercustgeo_aggcity.sort_values('ca', ascending=False).ca_pct.head(10),
            labels=ordercustgeo_aggcity.sort_values(
    'ca', ascending=False).head(10).index, normalize=False, autopct='%1.1f%%',
    shadow=True, startangle=90)  # explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title(
    'distribution des CA (cash/debit) par ville (les 10 plus representées)')
plt.show()

####################################################################
ordercustgeo_aggstate = ordercustgeo.loc[
    ordercustgeo.pay_type == 'credit_voucher', :].groupby(
    by="customer_state").agg({'pay_sum': ['sum']})
ordercustgeo_aggstate.columns = ordercustgeo_aggstate.columns.droplevel(0)
ordercustgeo_aggstate.columns = ['ca']
ordercustgeo_aggstate['ca_pct'] = ordercustgeo_aggstate.ca / \
    ordercustgeo_aggstate.ca.sum()

ordercustgeo_aggcity = ordercustgeo.loc[
    ordercustgeo.pay_type == 'credit_voucher', :].groupby(
    by="customer_city").agg({'pay_sum': ['sum']})
ordercustgeo_aggcity.columns = ordercustgeo_aggcity.columns.droplevel(0)
ordercustgeo_aggcity.columns = ['ca']
ordercustgeo_aggcity['ca_pct'] = ordercustgeo_aggcity.ca / \
    ordercustgeo_aggcity.ca.sum()


# display(ordercustgeo_aggstate.sort_values('ca',ascending=False))
# display(ordercustgeo.value_counts('geolocation_state',normalize=True))
# display(ordercustgeo.value_counts('geolocation_city',normalize=True))

f, axes = plt.subplots(1, 2)

axes[0].pie(ordercustgeo_aggstate.sort_values('ca',
                                              ascending=False).ca_pct.head(10),
            labels=ordercustgeo_aggstate.sort_values(
    'ca', ascending=False).head(10).index, normalize=False, autopct='%1.1f%%',
    shadow=True, startangle=90)
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title(
    'distribution des CA (credit/voucher) par etat (les 10 plus représentés)')

axes[1].pie(ordercustgeo_aggcity.sort_values(
    'ca', ascending=False).ca_pct.head(10),
    labels=ordercustgeo_aggcity.sort_values(
    'ca', ascending=False).head(10).index, normalize=False, autopct='%1.1f%%',
    shadow=True, startangle=90)  # explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title(
    'distribution des CA (credit/voucher) par ville (les 10 plus representées)')
plt.show()

# geo
* beaucoup de coordonnées pour la meme ville et le meme zip
* on peut imaginer que cette base a ete remplie avec la base ip clients
* dans ce cas : SP (sao paulo), MG (Minas Gerais - capitale belo horzonte), RJ (Rio de Janeiro), RS (Rio Grande do Sul capitale:Porto Alegre ) , PR (Paraná capitale: Curitiba) sont les etats les plus representés ce qui semble normal

In [ ]:
# print(geo.geolocation_lng.min())
# print(geo.geolocation_lng.max())
# print(geo.geolocation_lat.min())
# print(geo.geolocation_lat.max())

import matplotlib.image as img
geo.sort_values('geolocation_lng')
geo.loc[geo.geolocation_city == 'santa lucia do piai', :]

geo1 = geo.copy()
# nettoyage
# on enleve les positions hors du bresil aberrantes
geo1.loc[((geo1.geolocation_lng > -28) | (geo1.geolocation_lng < -75)
          | (geo1.geolocation_lat > 5) | (geo1.geolocation_lat < -34)),
         ['geolocation_lat', 'geolocation_lng']] = np.nan
geo1.loc[(geo.geolocation_city == 'são paulo'),
         ['geolocation_city']] = 'sao paulo'

datanull = 1-(geo1.isnull().sum()/len(geo1.index))
plt.figure(figsize=(10, 2))
ax = datanull.plot.barh(
    color="#8d19a9", title='Pourcentage de données par variable filtrée',
    xlabel='pourcentage de données')
ax.grid(zorder=0)
plt.show()

display(datanull)

brazil_img = img.imread('input/brazil.png')

###################################################
geo1a = geo1.geolocation_city.value_counts()
geo1aa = geo1.groupby(by="geolocation_city").agg(
    {'geolocation_lat': ['mean'], 'geolocation_lng': ['mean']})
geo1aa.columns = geo1aa.columns.droplevel(0)
geo1aa.columns = ['lat', 'lng']
geo1aaa = geo1aa.join(pd.DataFrame(geo1a))

geo1aaa.columns = ['lat', 'lng', 'nb']
geo1aaa['pct'] = geo1aaa.nb/geo1aaa.nb.sum()
geo1aaa = geo1aaa.sort_values('nb', ascending=False).head(10)


geo1b = geo1.geolocation_state.value_counts()
geo1bb = geo1.groupby(by="geolocation_state").agg(
    {'geolocation_lat': ['mean'], 'geolocation_lng': ['mean']})
geo1bb.columns = geo1bb.columns.droplevel(0)
geo1bb.columns = ['lat', 'lng']
geo1bbb = geo1bb.join(pd.DataFrame(geo1b))

geo1bbb.columns = ['lat', 'lng', 'nb']
geo1bbb['pct'] = geo1bbb.nb/geo1bbb.nb.sum()
geo1bbb = geo1bbb.sort_values('nb', ascending=False).head(10)

plt.rcParams["figure.figsize"] = [16, 6]
plt.rcParams["figure.autolayout"] = True

f, axes = plt.subplots(1, 3)
sns.scatterplot(data=geo1, x='geolocation_lng',
                y='geolocation_lat', size=0.5, alpha=0.5, ax=axes[0])
sns.scatterplot(data=geo1aaa, x="lng", y="lat", size="pct",
                hue="pct", sizes=(40, 400), alpha=1, ax=axes[1])
sns.scatterplot(data=geo1bbb, x="lng", y="lat", size="pct",
                hue="pct", sizes=(40, 400), alpha=1, ax=axes[2])
axes[0].set_title('base geo par zip')
axes[1].set_title('base geo pour les 10 villes les plus rep')
axes[2].set_title('base geo pour les 10 etats les plus rep')
im0 = axes[0].imshow(brazil_img, extent=[-75, -28, -34, 5],
                     alpha=0.8, aspect='auto')
im1 = axes[1].imshow(brazil_img, extent=[-75, -28, -34, 5],
                     alpha=0.8, aspect='auto')
im2 = axes[2].imshow(brazil_img, extent=[-75, -28, -34, 5],
                     alpha=0.8, aspect='auto')
plt.show()

#################################################
f, axes = plt.subplots(1, 2)

label1 = geo1aaa.sort_values('nb', ascending=False).head(10).index
size1 = geo1aaa.sort_values('nb', ascending=False).head(10)['pct']

axes[0].pie(size1, labels=label1, autopct='%1.1f%%',
            shadow=True, startangle=90, normalize=False)
# explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[0].axis('equal')
axes[0].set_title('distribution des villes')

label2 = geo1bbb.sort_values('nb', ascending=False).head(10).index
size2 = geo1bbb.sort_values('nb', ascending=False).head(10)['pct']

axes[1].pie(size2, labels=label2, autopct='%1.1f%%',
            shadow=True, startangle=90, normalize=False)
# explode=explode,
# Equal aspect ratio ensures that pie is drawn as a circle.
axes[1].axis('equal')
axes[1].set_title('distribution des etats')
plt.show()

# fusion totale
* note pour plus tard , on ne peut garder que les delivred qui represente 97%
* en general quand il y a plus de 1 sequence de paiement : on a affaire à du voucher mais ca reste rare et les autres cas sont anecdotiques 

In [ ]:
# il peut y avoir plusieurs echanges pour les review : on garde la finale
# display(order_review.loc[order_review.order_id=='8e17072ec97ce29f0e1f111e598b0c85'])

order_review = pd.read_csv('input/archive/olist_order_reviews_dataset.csv',
                           delimiter=',',
                           error_bad_lines=False,
                           low_memory=False)
order_review1 = doublonrem(order_review, 'order_review', 'order_id')
full1 = ordercust2.set_index('order_id').join(
    order_review1.set_index('order_id'))

# nettoyage
# *le *1 permet de transformer le boolean en 0/1
full1['rev_title_flag'] = full1.review_comment_title.notna()*1
full1['rev_comment_flag'] = full1.review_comment_message.notna()*1

#################################################

h1 = full1.review_score.value_counts(normalize=True, dropna=False)
h2 = full1.rev_title_flag.value_counts(normalize=True, dropna=False)
h3 = full1.rev_comment_flag.value_counts(normalize=True, dropna=False)
display(full1.order_status.value_counts(normalize=True, dropna=False))

f, axes = plt.subplots(1, 3)
axes[0].pie(h1, labels=h1.index, autopct='%.0f%%', normalize=False)
axes[1].pie(h2, labels=h2.index, autopct='%.0f%%', normalize=False)
axes[2].pie(h3, labels=h3.index, autopct='%.0f%%', normalize=False)

axes[0].set_title('distribution des scores review')
axes[1].set_title("Presence d'un titre de review")
axes[2].set_title("Presence d'un commentaire de review")

plt.show()

f, axes = plt.subplots(1, 3)
sns.boxplot(x='review_score', y='rev_title_flag', data=full1,
            showmeans=True, showfliers=False, ax=axes[0])
sns.boxplot(x='review_score', y='rev_comment_flag',
            data=full1,
            showmeans=True,
            showfliers=False,
            meanprops={"marker": "o",
                       "markerfacecolor": "white",
                       "markeredgecolor": "black",
                       "markersize": "10"}, ax=axes[1])
sns.boxplot(x='order_status', y='rev_comment_flag',
            data=full1,
            showmeans=True,
            showfliers=False,
            meanprops={"marker": "o",
                       "markerfacecolor": "white",
                       "markeredgecolor": "black",
                       "markersize": "10"}, ax=axes[2])
axes[0].set_title("titre de review moyen par score review")
axes[1].set_title("commentaire de review moyen par score review")
axes[2].set_title("commentaire de review moyen par order_status")
axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=45)
plt.show

#################################################

listodr = ['review_id', 'review_creation_date', 'review_answer_timestamp',
           'review_comment_title', 'review_comment_message', 'rev_title_flag',
           'order_approved_at', 'order_delivered_carrier_date',
           'order_delivered_customer_date', 'order_estimated_delivery_date',
           'pay_ins_min', 'pay_ins_mean', 'pay_val_min', 'pay_val_max',
           'voucher']
full1 = full1.drop(columns=listodr)

pd.set_option('max_columns', None)
display(full1)
pd.set_option('max_columns', 10)

* 90% des commandes ne portent que sur 1 seul article
* 7.5% des commandes portant sur 2 ou plus articles portent sur le meme article
* 9.5% des commandes portant sur 2 ou plus articles portent sur la meme categorie
* on peut se debarasser des commandes multiples à categorie differente

In [ ]:
# volume pour synthetiser les dimensions
product5['product_vol_m3'] = product5['product_length_cm'] * \
    product5['product_height_cm']*product5['product_width_cm']*1e-06


pd.set_option('max_columns', None)
display(product5.loc[product5.order_id == 'e332eda82fe7ca4e86daf4f71ca785df'])
display(product5.loc[product5.order_id == '0097f0545a302aafa32782f1734ff71c'])
display(product5.loc[product5.order_id == '00bcee890eba57a9767c7b5ca12d3a1b'])
pd.set_option('max_columns', 10)

# aggregation par order_id
product5['lcat'] = product5.product_category_name_english.str.len()
prodagg = product5.groupby(by="order_id").agg({'order_item_id': ['count', 'max'],
                                               'price': ['sum', 'mean', 'std'],
                                              'freight_value': ['sum', 'mean'],
                                               'product_weight_g': ['sum', 'mean'],
                                               'lcat': ['std']})

prodagg.columns = prodagg.columns.droplevel(0)
prodagg.columns = ['it_count', 'it_max', 'pr_sum', 'pr_mean',
                   'pr_std', 'fdp_sum', 'fdp_mean', 'poi_sum', 'poi_mean', 'lcat_std']

# vision du pourcentage de commandes par nombre d'articles
p1 = pd.DataFrame(prodagg.it_count.value_counts(normalize=True, dropna=False))
p1.columns = ['pct']
p2 = p1.loc[p1.index <= 3, :]
# display(p2)

f, axes = plt.subplots(1, 2)
axes[0].pie(p2.pct, labels=p2.index, autopct='%.0f%%', normalize=False)
axes[0].set_title("distribution des commandes multiples (nb d'articles)")

sns.boxplot(x='it_count', y='pr_sum', data=prodagg.loc[prodagg.it_count <= 3],
            showmeans=True,
            showfliers=False,
            meanprops={"marker": "o",
                       "markerfacecolor": "white",
                       "markeredgecolor": "black",
                       "markersize": "10"}, ax=axes[1])
axes[1].set_title("panier moyen en fonciton du nombre d'article par commande")

#####################################################
# vision des commmandes à articles differents ('variation de prix des articles')
display(prodagg.pr_std.value_counts(normalize=True, dropna=False))
# vision des commmandes à catégories differentes
#('variation du nombre de lettre des catégories')
display(prodagg.lcat_std.value_counts(normalize=True, dropna=False))

#####################################################
# on retire les dimensions car resumées en volume
product6 = product5.copy()
list2dr = ['product_length_cm', 'product_height_cm', 'product_width_cm']
product6 = product6.drop(columns=list2dr)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import normaltest
from scipy.stats import shapiro
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
product6 = product5.copy()
prodagg2 = product6.groupby(by="product_category_name_english").agg({
    'price': ['count', 'median', 'std'],
    'freight_value': ['median', 'std'],
    'product_weight_g': ['median', 'std'],
    'product_length_cm': ['median', 'std'],
    'product_height_cm': ['median', 'std'],
    'product_width_cm': ['median', 'std'],
    'product_vol_m3': ['median', 'std']})
prodagg2.columns = prodagg2.columns.droplevel(0)
prodagg2.columns = ['count', 'pr', 'pr_std', 'fdp', 'fdp_std', 'poi',
                    'poi_std', 'l', 'l_std', 'h', 'h_std', 'w', 'w_std',
                    'vol', 'vol_std']

# normalisation de l'erreur
# display(prodagg2)
prodagg2.pr_std = prodagg2.pr_std/prodagg2.pr
prodagg2.vol_std = prodagg2.vol_std/prodagg2.vol
# display(prodagg2)

prodagg3 = prodagg2[['count', 'pr', 'pr_std',
                     'fdp', 'poi', 'vol', 'vol_std']].copy()
pd.set_option('max_rows', None)
# display(prodagg3)
pd.set_option('max_rows', 10)

# Tansformation MINMAX scaler du jeu pour standardiser les données
# on utlise MINMAX car bcp de 0 et de 1
scaler = MinMaxScaler()
scaler.fit(prodagg3)

prodagg3_MM = pd.DataFrame(scaler.transform(
    prodagg3), columns=prodagg3.columns, index=prodagg3.index)
# display(prodagg3_MM)

#################################
# normalité

# https://towardsdatascience.com/pearson-coefficient-of-correlation-explained-369991d93404


def calculate_corr(df):
    """calcul des correlation."""

    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pearscoeff = dfcols.transpose().join(dfcols, how='outer')
    pearspvalues = pearscoeff.copy()
    shappvalues = pearscoeff.copy()
    normpvalues = pearscoeff.copy()
    spearcoeff = pearscoeff.copy()
    spearpvalues = pearscoeff.copy()
    for r in df.columns:
        for c in df.columns:
            pearscoeff[r][c] = round(pearsonr(df[r], df[c])[0], 4)
            pearspvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
            spearcoeff[r][c] = round(spearmanr(df[r], df[c])[0], 4)
            spearpvalues[r][c] = round(spearmanr(df[r], df[c])[1], 4)
            if r == c:
                # attention pvalue shapiro wilk seulement pour N<5000
                shappvalues[r][c] = round(shapiro(df[r])[1], 4)
                # d'agostino basé sur omnibus plus adapté pour
                # les grands echantillons
                normpvalues[r][c] = round(normaltest(df[r])[1], 4)
    pearscoeff[df.columns] = pearscoeff[df.columns].astype(float)
    pearspvalues[df.columns] = pearspvalues[df.columns].astype(float)
    shappvalues[df.columns] = shappvalues[df.columns].astype(float)
    normpvalues[df.columns] = normpvalues[df.columns].astype(float)
    spearcoeff[df.columns] = spearcoeff[df.columns].astype(float)
    spearpvalues[df.columns] = spearpvalues[df.columns].astype(float)
    return pearscoeff, pearspvalues, shappvalues,\
        normpvalues, spearcoeff, spearpvalues


cocorr, pvcorr, shap, dagos, cospcorr, pvspcorr = calculate_corr(prodagg3_MM)

plt.rcParams["figure.figsize"] = [16, 6]
f, axes = plt.subplots(1, 2)

sns.heatmap(
    cocorr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True,
    annot=True,
    fmt='.2f',
    linewidths=.5, ax=axes[0])
axes[0].title.set_text('Coefficient correlation de Pearson')

sns.heatmap(
    pvcorr,
    vmin=0, vmax=1, center=0.5,
    cmap=sns.diverging_palette(200, 20, n=200),
    square=True,
    annot=True,
    fmt='.2f',
    linewidths=.5, ax=axes[1])
axes[1].title.set_text('Pvalue correlation de Pearson')

plt.show()


# scipy.cluster.hierarchy.dendrogram(
# Z, (linkage matrix)
# p=30, (p parameter for truncate_mode)
# truncate_mode=None, (Truncation is used to condense the dendrogram)
#defaut: none
# lastp
# level!No more than p levels of the dendrogram tree are displayed
# color_threshold=None,
# get_leaves=True,
#orientation='top', bottom, left, right
# labels=None,
# default, labels is None so the index of the original observation
# is used to label the leaf nodes
# labels[i] value is the text to put under the th leaf node only if
# it corresponds to an original observation
# count_sort=False,
# distance_sort=False,
# show_leaf_counts=True,
# no_plot=False,
# no_labels=False,
# leaf_font_size=None,
# leaf_rotation=None,
# leaf_label_func=None,
# show_contracted=False,
# link_color_func=None,
# ax=None,
# above_threshold_color='C0')

Z1 = linkage(prodagg3_MM.loc[:, ['pr', 'vol']], 'ward')
Z2 = linkage(prodagg3_MM.loc[:, ['pr', 'vol', 'pr_std', 'vol_std']], 'ward')
Z3 = linkage(prodagg3_MM.loc[:, ['count', 'pr', 'vol']], 'ward')
Z4 = linkage(
    prodagg3_MM.loc[:, ['count', 'pr', 'vol', 'pr_std', 'vol_std']], 'ward')

plt.rcParams["figure.figsize"] = [16, 10]
f, axes = plt.subplots(1, 2)
dendrogram(Z1, orientation='right', labels=prodagg3_MM.index,
           leaf_font_size=10, ax=axes[0])
dendrogram(Z2, orientation='right', labels=prodagg3_MM.index,
           leaf_font_size=10, ax=axes[1])
axes[0].set_title("HClustering Dendrogram: features 'pr','vol'")
axes[1].set_title(
    "HClustering Dendrogram: features 'pr','vol','pr_std','vol_std'")
plt.show()
f, axes = plt.subplots(1, 2)
dendrogram(Z3, orientation='right', labels=prodagg3_MM.index,
           leaf_font_size=10, ax=axes[0])
dendrogram(Z4, orientation='right', labels=prodagg3_MM.index,
           leaf_font_size=10, ax=axes[1])
axes[0].set_title("HClustering Dendrogram: features 'count','pr','vol'")
axes[1].set_title(
    "HClustering Dendrogram: features 'count','pr','vol','pr_std','vol_std'")
plt.show()

#######################################################

In [ ]:
# cluster2
indA2 = ['computers', 'furniture_bedroom',
         'furniture_mattress_and_upholstery', 'home_appliances_2',
         'industry_commerce_and_business',
         'office_furniture', 'small_appliances_home_oven_and_coffee']

indB2 = ['agro_industry_and_commerce', 'air_conditioning',
         'art', 'arts_and_craftmanship', 'audio',
         'bed_bath_table', 'books_general_interest', 'books_imported',
         'books_technical', 'cds_dvds_musicals',
         'christmas_supplies', 'computers_accessories',
         'construction_tools_safety',
         'cool_stuff', 'costruction_tools_garden',
         'diapers_and_hygiene', 'drinks', 'fashion_childrens_clothes',
         'fashion_male_clothing', 'fashion_shoes',
         'fashion_underwear_beach', 'flowers', 'food', 'food_drink',
         'furniture_decor', 'furniture_living_room', 'garden_tools',
         'home_comfort_2', 'home_confort', 'housewares',
         'kitchen_dining_laundry_garden_furniture', 'la_cuisine',
         'luggage_accessories', 'perfumery', 'pet_shop',
         'security_and_services', 'stationery', 'tablets_printing_image',
         'toys', 'watches_gifts']

indC2 = ['auto', 'baby', 'cine_photo', 'consoles_games',
         'construction_tools_construction',
         'construction_tools_lights',
         'costruction_tools_tools', 'dvds_blu_ray', 'electronics',
         'fashio_female_clothing',
         'fashion_bags_accessories',
         'fashion_sport', 'fixed_telephony', 'health_beauty',
         'home_appliances',
         'home_construction', 'market_place',
         'music', 'musical_instruments', 'party_supplies',
         'signaling_and_security',
         'small_appliances', 'sports_leisure', 'telephony']

prodagg3.loc[indA2, ['GR2']] = 'A'
prodagg3.loc[indB2, ['GR2']] = 'B'
prodagg3.loc[indC2, ['GR2']] = 'C'

prodagg3_GR2 = prodagg3.groupby(by="GR2").agg({'count': ['mean'],
                                              'pr': ['median'],
                                               'pr_std': ['median'],
                                               'vol': ['median'],
                                               'vol_std': ['median']})
prodagg3_GR2.columns = prodagg3_GR2.columns.droplevel(0)
prodagg3_GR2.columns = ['count', 'pr', 'pr_std', 'vol', 'vol_std']
display(prodagg3_GR2)

# cluster4
indA4 = ['arts_and_craftmanship', 'audio', 'auto', 'baby',
         'books_general_interest',
         'books_imported', 'books_technical',
         'cds_dvds_musicals', 'christmas_supplies', 'consoles_games',
         'construction_tools_construction',
         'construction_tools_safety', 'costruction_tools_garden',
         'cool_stuff',
         'costruction_tools_tools',
         'diapers_and_hygiene', 'drinks', 'electronics',
         'fashion_childrens_clothes',
         'fashion_male_clothing',
         'fashion_shoes', 'fashion_underwear_beach', 'fixed_telephony',
         'flowers',
         'food', 'food_drink', 'garden_tools',
         'home_appliances', 'home_construction', 'market_place', 'music',
         'musical_instruments',
         'party_supplies', 'perfumery',
         'pet_shop', 'security_and_services', 'small_appliances',
         'stationery',
         'tablets_printing_image', 'telephony', 'toys']

indB4 = ['cine_photo', 'construction_tools_lights', 'dvds_blu_ray',
         'fashio_female_clothing',
         'fashion_bags_accessories',
         'fashion_sport', 'signaling_and_security']

indC4 = ['agro_industry_and_commerce', 'air_conditioning', 'art',
         'computers', 'furniture_bedroom',
         'furniture_living_room',
         'furniture_mattress_and_upholstery', 'home_appliances_2',
         'home_comfort_2', 'home_confort',
         'industry_commerce_and_business',
         'kitchen_dining_laundry_garden_furniture', 'la_cuisine',
         'luggage_accessories', 'office_furniture',
         'small_appliances_home_oven_and_coffee']

indD4 = ['bed_bath_table', 'computers_accessories',
         'furniture_decor', 'health_beauty',
         'housewares', 'sports_leisure',
         'watches_gifts']

prodagg3.loc[indA4, ['GR4']] = 'A'
prodagg3.loc[indB4, ['GR4']] = 'B'
prodagg3.loc[indC4, ['GR4']] = 'C'
prodagg3.loc[indD4, ['GR4']] = 'D'

prodagg3_GR4 = prodagg3.groupby(by="GR4").agg({'count': ['mean'],
                                              'pr': ['median'],
                                               'pr_std': ['median'],
                                               'vol': ['median'],
                                               'vol_std': ['median']})
prodagg3_GR4.columns = prodagg3_GR4.columns.droplevel(0)
prodagg3_GR4.columns = ['count', 'pr', 'pr_std', 'vol', 'vol_std']
display(prodagg3_GR4)

plt.rcParams["figure.figsize"] = [16, 6]
plt.rcParams["figure.autolayout"] = True

f, axes = plt.subplots(1, 2)
sns.scatterplot(data=prodagg3, x="vol", y="pr", hue='GR2', ax=axes[0])
sns.scatterplot(data=prodagg3, x="vol_std", y="pr_std", hue='GR2', ax=axes[1])
axes[0].set_title('product_category par prix/volume et par cluster GR2')
axes[1].set_title(
    'product_category par prix std/volume std et par cluster GR2')
plt.show()

f, axes = plt.subplots(1, 2)
sns.scatterplot(data=prodagg3, x="vol", y="pr", hue='GR4', ax=axes[0])
sns.scatterplot(data=prodagg3, x="count", y="pr", hue='GR4', ax=axes[1])
axes[0].set_title('product_category par prix/volume et par cluster GR4')
axes[1].set_title(
    'product_category par prix std/volume std et par cluster GR4')
plt.show()

In [ ]:
pd.set_option('max_columns', None)
display(product5)
display(product5.loc[product5.order_item_id == 6])
display(product5.loc[product5.order_id == 'ab14fdcfbe524636d65ee38360e22ce8'])
pd.set_option('max_columns', 10)

# cas d'une commande à plusieurs articles et un paiement credit en 4 fois

In [ ]:
display(order_payment)
display(order.loc[order.order_id == 'ab14fdcfbe524636d65ee38360e22ce8'])
display(order_item.loc[order_item.order_id ==
        'ab14fdcfbe524636d65ee38360e22ce8'])
display(product.loc[product.product_id == '9571759451b1d780ee7c15012ea109d4'])
display(order_payment.loc[order_payment.order_id ==
        'ab14fdcfbe524636d65ee38360e22ce8'])

# cas d'une commande àvec sequence >1

In [ ]:
pay1.loc[pay1.pay_seq_count > 1]
# 009ac365164f8e06f59d18a08045f6c4
# 0016dfedd97fc2950e388d2971d718c7

display(order.loc[order.order_id == '009ac365164f8e06f59d18a08045f6c4'])
display(order_item.loc[order_item.order_id ==
        '009ac365164f8e06f59d18a08045f6c4'])
display(product.loc[product.product_id == '35557c68a22ecebcf066e25ca2ddc144'])
display(order_payment.loc[order_payment.order_id ==
        '009ac365164f8e06f59d18a08045f6c4'])
display(order_payment.loc[order_payment.order_id ==
        '009ac365164f8e06f59d18a08045f6c4', ['payment_value']].sum())

In [ ]:
display(order_payment.loc[order_payment.payment_sequential > 1].value_counts(
    'payment_type'))
display(order_payment.loc[(order_payment.payment_sequential > 1) & (
    order_payment.payment_type == 'debit_card')])
display(order_payment.loc[order_payment.order_id ==
        'a4431cbd79dbddaae7988ce6091cbc3c'])